# 加载函数

In [ ]:
from common import read
from common import plot_df, plot_district, plot
import pandas as pd
def plotCity(df):
    gp = df.groupby(['成交时间'])['成交价(元/平)']
    res=pd.DataFrame({"volume":gp.size(),"median_price":gp.median(), "mean_price":gp.mean()})
    res = res.iloc[:len(res),:]
    title = city
    res = plot(res, city, title, MA, ma_length, start_date)
    return res
def plotAllDistrict(df):
    districts = list(df['下辖区'].unique())
    res = {}
    for district in districts:
        if str(district) != 'nan':
            res[district] = plot_district(df, city, district, ma_length, start_date)
    return res

# 画各个城市的趋势图

In [ ]:
import os
MA = True
ma_length = 30
start_date = '2015-01-01'

cityList = ['北京', '上海', '深圳', '杭州', '广州', '长沙', '厦门', '宁波', '合肥', '成都','重庆','武汉',
            '西安','石家庄','苏州','南京', '大连', '青岛', '无锡', '保定', '温州', '廊坊', '天津']
#cityList = ['北京', '上海','深圳']
cityList = ['天津']
data = {}
res = {}
districtRes = {}
for city in cityList:
    print(city)
    df = read(city)
    data[city] = df
    res[city] = plotCity(df)
    districtRes[city]=plotAllDistrict(df)

In [ ]:
df['下辖区'].unique()

In [ ]:
def drawDown(res):
    dd = {}
    for district in res:
        try:
            dd[district] = (res[district]['mean_price'][-1]/res[district]['mean_price'].max()-1)
        except:
            pass
    dd = pd.DataFrame({'跌幅':dd}).sort_values('跌幅')
    dd['跌幅'] = ["%.2f%%"%(a*100) for a in dd['跌幅']]
    display(dd)
#display(districtRes['北京']['海淀'])
drawDown(districtRes['北京'])

In [ ]:
#计算城市排名
if not os.path.exists('fig/allcity'):
    os.makedirs('fig/allcity')
os.system('rm fig/allcity/*')
median = {}
mean = {}
yearChange = {}
change = {}
monthChange = {}
for city in cityList:
    median[city] = int(res[city]['median_price'][-1])
    mean[city] = int(res[city]['mean_price'][-1])
    try:
        yearChange[city] = "%.2f%%"%(100 * (res[city]['median_price'][-1]/res[city]['median_price'][-365] - 1))
    except:
        yearChange[city] = '数据不足'
    change[city] = "%.2f%%"%(100 * (res[city]['median_price'][-1]/res[city]['median_price'][-180] - 1))
    monthChange[city] = "%.2f%%"%(100 * (res[city]['median_price'][-1]/res[city]['median_price'][-30] - 1))

cityRank = pd.DataFrame({'中位数':median, '均值':mean, 
                         '近一年':yearChange,
                         '近半年':change, '近一个月':monthChange}).sort_values('中位数', ascending = False)
cityRank['城市'] = cityRank.index
cityRank.index = range(1, len(cityRank) + 1)
cityRank = cityRank.loc[:,['城市', '中位数', '均值', '近一年', '近半年','近一个月']]
display(cityRank)
for index, row in cityRank.iterrows():
    city = row['城市']
    index = int(index)
    cmd = 'cp fig/%s/%s.png fig/allcity/%.2d.%s.png'%(city, city, index, city)
    os.system(cmd)

# 合并历史数据

In [ ]:
from common import read
cityList = ['北京', '上海', '深圳', '杭州', '广州', '长沙', '厦门', '宁波', '合肥', '成都','重庆','武汉',
            '西安','石家庄','苏州','南京', '大连', '温州']
for city in cityList:
    df = read(city)
    df.to_csv('data/all/%s.csv'%city, index=False)

In [ ]:
if not os.path.exists('data/res'):
    os.makedirs('data/res')
for city in cityList:
    res[city].to_csv('data/res/'+city+'.csv')

In [ ]:
from common import read
city = 'alltj'
df = read(city)
xiaoqu= open('xiaoqu/tjxiaoqu.txt', 'w')
xiaoquList = df['小区'].unique()
print(len(xiaoquList))
for xq in xiaoquList:
    if str(xq) != 'nan':
        xiaoqu.write(xq+'\n')
    
xiaoqu.close()

In [3]:
from common import read
city = 'alltj'
df = read(city)
#df.drop_duplicates(subset=['链家编号'])
print(df.columns)
print(len(df))
df.to_csv(city+'.csv')

raw count: 125880
count after drop duplicates 105280
count after drop less than 1000 105277
count after drop 40, 50 104100
Index(['状态', '成交价(元/平)', '建筑面积', '挂牌时间', '上次交易时间', '建成时间', '城市', '下辖区', '商圈',
       '小区', '所属环线', '户型', '朝向', '梯户比例', '房屋用途', '土地年限', '建筑类型', '交易权属', '装修',
       '建筑结构', '供暖方式', '产权所属', '户型结构', '电梯', '所在楼层', '房屋年限', '标题', '网址',
       '关注（人）', '带看（次）', '挂牌价格(万)', '成交周期(天)', '调价(次)', '浏览（次）', '套内面积',
       '成交时间', '售价(万)'],
      dtype='object')
104100


In [ ]:
df['土地年限']

In [ ]:
from common import plot_dfs
import pandas as pd
def plotDianti(df):
    pd.options.display.max_columns = None
    df_dt= df.loc[df['电梯'] == '有']
    df_ndt= df.loc[df['电梯'] != '有']
    print('有电梯', len(df_dt))
    print('无电梯',len(df_ndt))
    plot_dfs([df_ndt,df_dt], '%s有无电梯'%city, ['无电梯', '电梯'], 30, '2015-01-01')
for city in cityList:
    df = read(city)
    plotDianti(df)

In [ ]:
from common import plot_df
df_sjs = df.loc[df['下辖区']=='石景山']
xiaoquList = df_sjs['小区'].unique()
for xiaoqu in xiaoquList:
    plot_df(df_sjs.loc[df_sjs["小区"]== xiaoqu], "石景山", xiaoqu, True, 10)

In [ ]:
import pandas as pd
x=df_sjs.groupby('小区')

x_mean  = x.mean()
x_size = x.size()
x_mean = x_mean.merge(pd.DataFrame({'size':x_size}), left_index = True, right_index = True)
x_mean=x_mean.loc[x_size>=5]
x_mean=x_mean.sort_values(by='成交价(元/平)', ascending=False).loc[:,["建筑面积","成交价(元/平)","售价(万)", 'size']]
x_mean

In [ ]:
from common import plot, plot_dfs, plot_df
MA = True
ma_length = 10
def plot_xiaoqu(xiaoqu, df):
    df_xiaoqu = df.dropna(subset=['小区'])
    df_xiaoqu = df_xiaoqu.loc[df_xiaoqu['小区'].str.contains(xiaoqu)]
    #plot_dfs([df, df_xiaoqu], xiaoqu, ['全市', xiaoqu], ma_length, '2015-01-01')
    plot_df(df_xiaoqu, city, xiaoqu, MA, ma_length)
    
ma_length = 30
#plot_xiaoqu('八角', data['北京'])
plot_xiaoqu('观林园', df)

In [ ]:
pd.options.display.max_columns = None
#df.loc[df['小区'].str.contains('团结湖南里')]
df.loc[df['小区'].str.contains('爱乐')].sort_values(by='成交时间', ascending=False)
#df.loc[df['小区'].str.contains('平乐园')].sort_values(by='成交时间', ascending=False)

In [ ]:
x=df.groupby('小区')
x_mean  = x.mean()
x_size = x.size()
#x_size
x_mean=x_mean.loc[x_size>=1]
x_mean=x_mean.sort_values(by='成交价(元/平)', ascending=False).loc[:,["建筑面积","成交价(元/平)","售价(万)"]]
x_mean

In [ ]:
x_mean.index[:10]

In [ ]:
df.sort_values('售价(万)', ascending=False).loc[:,["小区", "建筑面积","成交价(元/平)", "售价(万)"]]

In [ ]:
df.sort_values('成交价(元/平)', ascending=False).loc[:,["小区", "建筑面积","成交价(元/平)", "售价(万)","成交时间"]]

In [ ]:
ma_length = 10
mean_price = df['成交价(元/平)'].mean()
price_std = df['成交价(元/平)'].std()
print('mean:', mean_price, 'std:', price_std)
threshold = 1.3
#high_df = df.loc[df['成交价(元/平)']>= mean_price + threshold * price_std]
#low_df =  df.loc[df['成交价(元/平)']< mean_price - threshold* price_std]
#medium_df = df.loc[df['成交价(元/平)']< mean_price + threshold * price_std]
#medium_df = medium_df.loc[medium_df['成交价(元/平)']>= mean_price - threshold * price_std]
sort_key = '成交价(元/平)'#
#sort_key = '售价(万)'
df = df.sort_values(sort_key, ascending = False)
count = len(df)//3
high_df = df.iloc[:count]
low_df = df.iloc[-count:]
medium_df = df.iloc[count:-count]
print(len(high_df), len(low_df), len(medium_df))
print(high_df[sort_key].mean(), medium_df[sort_key].mean(), low_df[sort_key].mean() )
ma_length = 30
def getPriceSeries(df):
    gp = df.groupby(['成交时间'])['成交价(元/平)']
    res=pd.DataFrame({"volume":gp.size(),"median_price":gp.median(), "mean_price":gp.mean()})
    res = res.sort_index()
    res = res.iloc[:len(res)-1]
    res = get_moving_average(res, ma_length)
    return res
highSeries=getPriceSeries(high_df)
mediumSeries=getPriceSeries(medium_df)
lowSeries=getPriceSeries(low_df)


In [ ]:
fig, ax = plt.subplots()
ax.plot(highSeries['mean_price']/highSeries['mean_price'][0])
ax.plot(mediumSeries['mean_price']/mediumSeries['mean_price'][0])
ax.plot(lowSeries['mean_price']/lowSeries['mean_price'][0])
plt.xticks(rotation=45)
ax.legend(['high=%.f yuan'%(high_df[sort_key].mean()),
           'medium=%.f yuan'%medium_df[sort_key].mean(),
           'low=%.f yuan'%low_df[sort_key].mean()])
xticks = ax.xaxis.get_major_ticks()
interval = len(xticks)// 10
ax.set_xticks(ax.get_xticks()[::interval])
'done'

In [ ]:
plt.axis

In [ ]:
plt.plot(highSeries['median_price']/highSeries['median_price'][0])
plt.plot(mediumSeries['median_price']/mediumSeries['median_price'][0])
plt.plot(lowSeries['median_price']/lowSeries['median_price'][0])
plt.xticks(rotation=90)
plt.legend(['high','medium', 'low'])


In [ ]:
def plotAllDistrict(df, ma_length = 10):
    districts = list(set(df['下辖区']))
    legend = ['beijing']
    data = []
    gp = df.groupby(['成交时间'])['成交价(元/平)']
    res=pd.DataFrame({"volume":gp.size(),"median_price":gp.median(), "mean_price":gp.mean()})
    res = res.iloc[:len(res)-1,:]
    res = get_moving_average(res, ma_length)

    data.append(res)
    for district in districts:
        gp = df.loc[df['下辖区']==district].groupby(['成交时间'])
        res = pd.DataFrame({'volume':gp.size(),'mean_price':gp['成交价(元/平)'].mean(), 'median_price':gp['成交价(元/平)'].median()})
        res = res.iloc[:len(res) -1,:]
        res = get_moving_average(res, ma_length)
        if len(res) < 1:
            continue
        data.append(res)
        title = pinyin(district)
        if district == '朝阳':
            title = 'chao yang'
        elif district == '长宁':
            title = 'chang ning'
        elif district == '闵行':
            title = 'min hang'
        else:
            title = " ".join([x[0] for x in title])
        legend.append(title)
   
    for i in range(len(data)):
        plt.plot(data[i]['mean_price']/data[i]['mean_price'].iloc[0])
    plt.xticks(rotation=90)
    plt.legend(legend)
plotAllDistrict(df, 30)
    

In [ ]:
city = 'alltj'
MA = True
ma_length = 30
start_date = '2015-01-01'
df = read(city)
res = plotCity(df)
plotAllDistrict(df)

In [9]:
from common import read
city = '天津'
df = read(city)
df.groupby('下辖区').size().sort_values()

raw count: 104100
count after drop duplicates 104100
count after drop less than 1000 104100
count after drop 40, 50 104100


下辖区
宁河           22
静海           38
海河教育园区      143
开发区         192
蓟州          341
宝坻          583
塘沽         2507
津南         2587
武清         2811
滨海新区       2867
和平         4312
北辰         4634
东丽         5512
红桥         5937
河北         6700
西青        10668
河东        12797
河西        16081
南开        25265
dtype: int64

In [8]:
sum(df['下辖区']=='天津')

0

In [11]:
[ a.split()[0] for a in '''和平         4312
北辰         4634
东丽         5512
红桥         5937
河北         6700
西青        10668
河东        12797
河西        16081
南开        25265'''.split('\n')]

['和平', '北辰', '东丽', '红桥', '河北', '西青', '河东', '河西', '南开']